In [76]:
print('hello supervised learning')

hello supervised learning


In [77]:
import polars as pl

## Load 

Load from CSV and clean up the data
1. Remove all rows where the year is 2014 because its the first year there are no pbs
2. Clean up gender as it contains "M" or "Male" and "F" or "Female". Just take the first character for simplity
3. Data Types

In [78]:
results_with_pb_df = pl.read_csv("data/withpb-all-results.csv")


results_with_pb_df = results_with_pb_df.drop("XC_Count_of_event", "XC_Position_Min", "XC_Position_Max", "XC_Min_Age", "XC_Max_Age","exists", "from_file")

# Gender Clean up
results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("Gender").str.slice(0, 1).alias("Gender")
)

results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("is_pb").cast(pl.Boolean),
    pl.col("first_time_run").cast(pl.Boolean)
)

display(results_with_pb_df)




Position,Time,Bib Number,Name,Gender,Gender Position,Club,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time
i64,str,i64,str,str,f64,str,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,"""00:29:14""",880,"""Andrew Masters""","""M""",3.0,"""Lewes AC""",10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:39.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,24.0,16.0,8.0,12.0,12.0,12.0,10.0,10.0,10.0,1.0,1.0,1.0,2014.0,2014.0,12.0,12.0
4,"""00:29:34""",588,"""Paul Cousins""","""M""",4.0,"""Haywards Heath Harriers""",10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:41.750000""",1.0,1.0,false,true,null,"""nan""",null,null,null,9.0,5.0,4.0,2.0,5.0,7.0,10.0,10.0,10.0,3.0,3.0,1.0,2014.0,2014.0,2.0,7.0
5,"""00:29:47""",902,"""Michael Neville""","""M""",5.0,"""Chichester Runners""",10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:43.375000""",1.0,1.0,false,true,null,"""nan""",null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
6,"""00:30:07""",918,"""Conrad Meagher""","""M""",6.0,"""Chichester Runners""",10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:45.875000""",2.0,2.0,false,true,null,"""nan""",null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
7,"""00:30:09""",530,"""John Russell""","""M""",7.0,"""Portslade Hedgehoppers""",10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:46.125000""",1.0,1.0,false,true,null,"""nan""",null,null,null,23.0,15.0,8.0,10.0,27.0,72.0,10.0,9.0,8.0,10.0,10.0,1.0,2014.0,2014.0,10.0,72.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",631,"""Ann Savidge""","""F""",303.0,"""Burgess Hill Runners""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:07:17.461774""",92.0,52.0,false,false,"""2015.0""","""00:38:16""","""571.0""","""down""","""09:25""",93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0
628,"""00:54:49""",613,"""Christine Jones""","""F""",304.0,"""Burgess Hill Runners""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:08:22.905199""",93.0,53.0,false,true,null,"""nan""",null,null,null,93.0,40.0,53.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
629,"""00:56:27""",429,"""Kalpna Robertson""","""F""",305.0,"""Lewes AC""",1.0,"""Worthing Lido""",2016,1,31,"""2016-01-31""",6.54,"""0:08:37.889908""",34.0,14.0,false,true,null,"""nan""",null,null,null,33.0,20.0,13.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null


## Do we have balanced target classes?

We calculate the PB from the prior runs. 
- The first year is 2014 which has no pbs because there is no prior data
- this needs to be removed as its unbalances the data

In [79]:
results_with_pb_df_all = results_with_pb_df

# Pivot the DataFrame
pivot_df = results_with_pb_df_all.pivot(
    values="is_pb",
    index="year",
    on="is_pb",
    aggregate_function="len"
).sort("year")


display(pivot_df)

# Remove all rows where the year is 2014
results_with_pb_df = results_with_pb_df.filter(pl.col("year") != 2014)


year,false,true
i64,u32,u32
2014,6811,null
2015,6289,1116
2016,5925,1283
2017,5263,1042
2018,5269,904
2019,5451,1172
2020,378,93
2022,3967,637
2023,3758,631


## Find Categoricals

These are columns with limited subset of values.

For example **education** could be 
- Graduate
- Not Graduate

In our data set
- Club 
- RaceName
- Gender
All of which are unlikely to influence a pb

However, I'll add of these are categorical variables to the model.
- RaceName
- Gender

The two `to_dummies` colum will turn these into columns with 1s and 0s.
- So for `RaceName` it will create a column for each unique value in the column and set it to 1 if the value is present and 0 if not.
- `RaceName_Downland Dash`
- `RaceName_Fittleworth`
- for each race combo





In [80]:
results_with_pb_with_dummies_df = results_with_pb_df.to_dummies(
    ["Gender", "RaceName"], drop_first=True
)

In [81]:
display(results_with_pb_with_dummies_df)

Position,Time,Bib Number,Name,Gender_F,Gender Position,Club,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time
i64,str,i64,str,u8,f64,str,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,str,f64,str,f64,f64,bool,bool,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,"""00:28:24""",328,"""Andy Massingale""",0,3.0,"""Henfield Joggers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,15.0,9.0,6.0,3.0,8.0,20.0,10.0,10.0,10.0,4.0,4.0,2.0,2014.0,2014.0,3.0,20.0
4,"""00:28:24""",280,"""Jon Bess""",0,4.0,"""Worthing Striders""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:33""",1.0,1.0,false,true,null,"""nan""",null,null,null,16.0,10.0,6.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
5,"""00:28:34""",461,"""Tiarnan Cotter""",0,5.0,"""Horsham Joggers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:34.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,33.0,14.0,19.0,15.0,19.0,23.0,10.0,10.0,10.0,2.0,2.0,2.0,2014.0,2015.0,15.0,23.0
6,"""00:28:50""",254,"""Paul Sargent""",0,6.0,"""Burgess Hill Runners""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:36.250000""",1.0,1.0,false,true,null,"""nan""",null,null,null,47.0,28.0,19.0,6.0,7.0,8.0,10.0,10.0,10.0,2.0,3.0,2.0,2014.0,2014.0,6.0,54.0
7,"""00:29:11""",167,"""Paul Cousins""",0,7.0,"""Haywards Heath Harriers""",10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:38.875000""",1.0,1.0,true,false,"""2014.0""","""00:29:34""","""4.0""","""up""","""00:23""",13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,2015.0,2014.0,2.0,132.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",631,"""Ann Savidge""",1,303.0,"""Burgess Hill Runners""",1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",6.54,"""0:07:17.461774""",92.0,52.0,false,false,"""2015.0""","""00:38:16""","""571.0""","""down""","""09:25""",93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0
628,"""00:54:49""",613,"""Christine Jones""",1,304.0,"""Burgess Hill Runners""",1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",6.54,"""0:08:22.905199""",93.0,53.0,false,true,null,"""nan""",null,null,null,93.0,40.0,53.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
629,"""00:56:27""",429,"""Kalpna Robertson""",1,305.0,"""Lewes AC""",1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",6.54,"""0:08:37.889908""",34.0,14.0,false,true,null,"""nan""",null,null,null,33.0,20.0,13.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null


## Split the data

Remove the target column from the features. **is_pb**

Features are the columns that will be used to predict the target variable. In this case, the features are all the columns except the is_pb column. 


In [82]:
pb_achieved_features_df = results_with_pb_with_dummies_df.drop("is_pb")

# drop non feature columns 
pb_achieved_features_df = pb_achieved_features_df.drop("Bib Number", "Name", "date","Club" )
pb_achieved_features_df = pb_achieved_features_df.drop( "Time", "prior_quickest_run_time", "pace" )
pb_achieved_features_df = pb_achieved_features_df.drop( "prior_quickest_run_in_year", "prior_quickest_run_position","prior_time_vs_current_change", "prior_time_vs_current_time" )

display(pb_achieved_features_df)

pb_achieved_target_df = results_with_pb_df.select("is_pb")

Position,Gender_F,Gender Position,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,distance,club_position,club_position_gender,first_time_run,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,runs_in_wsfrl,years_in_wsfrl,year_with_best_position,year_with_worst_position,best_position_all_time,worst_position_all_time
i64,u8,f64,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,f64,f64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3,0,3.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,15.0,9.0,6.0,3.0,8.0,20.0,10.0,10.0,10.0,4.0,4.0,2.0,2014.0,2014.0,3.0,20.0
4,0,4.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,16.0,10.0,6.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
5,0,5.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,33.0,14.0,19.0,15.0,19.0,23.0,10.0,10.0,10.0,2.0,2.0,2.0,2014.0,2015.0,15.0,23.0
6,0,6.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,true,47.0,28.0,19.0,6.0,7.0,8.0,10.0,10.0,10.0,2.0,3.0,2.0,2014.0,2014.0,6.0,54.0
7,0,7.0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,8.0,1.0,1.0,false,13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,2015.0,2014.0,2.0,132.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,1,303.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,92.0,52.0,false,93.0,40.0,53.0,341.0,441.0,571.0,4.0,2.0,1.0,7.0,10.0,2.0,2014.0,2015.0,341.0,613.0
628,1,304.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,93.0,53.0,true,93.0,40.0,53.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null
629,1,305.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,6.54,34.0,14.0,true,33.0,20.0,13.0,null,null,null,null,null,null,null,0.0,0.0,null,null,null,null


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 123

(
    pb_achieved_feature_train_df,
    pb_achieved_feature_test_df,
    pb_achieved_target_train_df,
    pb_achieved_target_test_df,
) = train_test_split(
    pb_achieved_features_df,
    pb_achieved_target_df,
    test_size=0.30,
    random_state=SEED,
)



## Scale the data

It is important to scale the data so that the model is not distorted by really large values.

### My notes are 
- Dimensionality reduction, which 
1. Rotates the data. Some like 45 degrees
2. Keeps the same data but reduce the complexity
- Principal Component Analysis (PCA) is a technique that can be used to reduce the dimensionality of the data.

### Notes from Copilot
The StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

In [84]:
scaler = StandardScaler()

pb_achieved_feature_train_scaled_df = scaler.fit_transform(
    pb_achieved_feature_train_df
)

pb_achieved_feature_test_scaled_df = scaler.fit_transform(
    pb_achieved_feature_test_df
)

Do we have balanced target classes?

# Logistic regression example

```
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```

## Constraints

> ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values


One common strategy is to use the mean or median of the column to fill in the missing values. Here is how you can do it using SimpleImputer from sklearn.impute:  
Import SimpleImputer.
Create an imputer instance with the desired strategy (e.g., mean, median).
Fit the imputer on the training data and transform both the training and test data.

```
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Create an imputer instance with the mean strategy
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the training data and transform both the training and test data
pb_achieved_feature_train_scaled_df = imputer.fit_transform(pb_achieved_feature_train_scaled_df)
pb_achieved_feature_test_scaled_df = imputer.transform(pb_achieved_feature_test_scaled_df)

# Create and fit the logistic regression classifier
logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```





In [85]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [93]:
# Convert Polars DataFrame to Pandas DataFrame
pb_achieved_feature_train_df_pandas = pb_achieved_feature_train_df.to_pandas()
pb_achieved_target_train_df_pandas = pb_achieved_target_train_df.to_pandas()
pb_achieved_feature_test_df_pandas = pb_achieved_feature_test_df.to_pandas()
pb_achieved_target_test_df_pandas = pb_achieved_target_test_df.to_pandas()



In [94]:
# Define the columns
# "Position", "Gender Position", 
low_value_good_columns = ["best_position_year_to_date", "average_position_year_to_date", "worst_position_year_to_date", "best_position_all_time", "worst_position_all_time"]
# low_value_good_columns = ["Position"]
high_value_good_columns = ["fellow_club_mates_running", "fellow_club_mates_running_male", "fellow_club_mates_running_female", "average_points_year_to_date", "best_points_year_to_date", "worst_points_year_to_date", "runs_in_wsfrl_year_to_date", "runs_in_wsfrl", "years_in_wsfrl"]
# high_value_good_columns = ["fellow_club_mates_running"]

# Check if columns exist in the DataFrame
missing_low_value_good_columns = [col for col in low_value_good_columns if col not in pb_achieved_feature_train_df_pandas.columns]
missing_high_value_good_columns = [col for col in high_value_good_columns if col not in pb_achieved_feature_train_df_pandas.columns]

# Print the results
if missing_low_value_good_columns:
    print(f"Missing columns in low_value_good_columns: {missing_low_value_good_columns}")
else:
    print("All columns in low_value_good_columns exist in the DataFrame.")

if missing_high_value_good_columns:
    print(f"Missing columns in high_value_good_columns: {missing_high_value_good_columns}")
else:
    print("All columns in high_value_good_columns exist in the DataFrame.")

All columns in low_value_good_columns exist in the DataFrame.
All columns in high_value_good_columns exist in the DataFrame.


In [97]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# Create imputers for different columns
# low_value_imputer = SimpleImputer(strategy="constant", fill_value=np.inf)
low_value_imputer = SimpleImputer(strategy="constant", fill_value=400)
high_value_imputer = SimpleImputer(strategy="constant", fill_value=0) #-np.inf is too big for a float 

# Combine the imputers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("low_value_good", low_value_imputer, low_value_good_columns),
        ("high_value_good", high_value_imputer, high_value_good_columns),
        # ("remaining", SimpleImputer(strategy="constant", fill_value=-1), "passthrough")
    ]
)

# Create a pipeline with the preprocessor, scaler, and the logistic regression classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("scaler", StandardScaler()),
    ("classifier", LogisticRegression(penalty=None))
])

# Fit the pipeline on the training data
pipeline.fit(pb_achieved_feature_train_df_pandas, pb_achieved_target_train_df_pandas["is_pb"])

# 
# # Transform the test data
# pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('low_value_good',
                                                  SimpleImputer(fill_value=400,
                                                                strategy='constant'),
                                                  ['best_position_year_to_date',
                                                   'average_position_year_to_date',
                                                   'worst_position_year_to_date',
                                                   'best_position_all_time',
                                                   'worst_position_all_time']),
                                                 ('high_value_good',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['fellow_club_mates_running',
                                                   'fellow_club_mates_running_male',
                                                   'fellow_club_mates_running_female',
                                                   'average_points_year_to_date',
                                                   'best_points_year_to_date',
                                                   'worst_points_year_to_date',
                                                   'runs_in_wsfrl_year_to_date',
                                                   'runs_in_wsfrl',
                                                   'years_in_wsfrl'])])),
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(penalty=None))])

In [98]:
pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)

AttributeError: This 'Pipeline' has no attribute 'transform'

In [107]:
# Transform the test data using the preprocessor part of the pipeline
pb_achieved_feature_train_preprocessed_df = pipeline.named_steps['preprocessor'].transform(pb_achieved_feature_train_df_pandas)

# Scale the preprocessed test data
pb_achieved_feature_test_scaled_df = pipeline.named_steps['scaler'].transform(pb_achieved_feature_train_preprocessed_df)

In [109]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_test_scaled_df,
    pb_achieved_target_train_df_pandas["is_pb"],
)

LogisticRegression(penalty=None)

In [110]:


logistic_regression_classifier.classes_



array([False,  True])

In [121]:
pb_achieved_feature_train_df_pandas[["best_position_year_to_date", "average_position_year_to_date", "worst_position_year_to_date", "best_position_all_time", "worst_position_all_time","fellow_club_mates_running", "fellow_club_mates_running_male", "fellow_club_mates_running_female", "average_points_year_to_date", "best_points_year_to_date", "worst_points_year_to_date", "runs_in_wsfrl_year_to_date", "runs_in_wsfrl", "years_in_wsfrl"]].columns

Index(['best_position_year_to_date', 'average_position_year_to_date',
       'worst_position_year_to_date', 'best_position_all_time',
       'worst_position_all_time', 'fellow_club_mates_running',
       'fellow_club_mates_running_male', 'fellow_club_mates_running_female',
       'average_points_year_to_date', 'best_points_year_to_date',
       'worst_points_year_to_date', 'runs_in_wsfrl_year_to_date',
       'runs_in_wsfrl', 'years_in_wsfrl'],
      dtype='object')

In [122]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": pb_achieved_feature_train_df_pandas[["best_position_year_to_date", "average_position_year_to_date", "worst_position_year_to_date", "best_position_all_time", "worst_position_all_time","fellow_club_mates_running", "fellow_club_mates_running_male", "fellow_club_mates_running_female", "average_points_year_to_date", "best_points_year_to_date", "worst_points_year_to_date", "runs_in_wsfrl_year_to_date", "runs_in_wsfrl", "years_in_wsfrl"]].columns,
            "coefficient": logistic_regression_classifier.coef_[0],
        }
    )
    .sort("coefficient", descending=True)
    .plot.bar(
        x="coefficient",
        y=alt.Y("feature", sort="-x"),
    )
    .properties(
        width=500,
        height=300,
    )
)

alt.Chart(...)

In [118]:
logistic_regression_classifier.coef_

array([[ 0.41634425,  0.06934528, -0.5019998 , -0.88662126,  0.61343622,
         0.05514941,  0.1124353 , -0.01849828,  0.05795886,  0.06848512,
        -0.07718903,  0.63145624, -0.51008773,  0.15290009]])

In [105]:

# Make predictions on the test data
predictions = pipeline.named_steps['classifier'].predict(pb_achieved_feature_test_scaled_df)

# Print the predictions
print(predictions)

[False False False ... False False False]


In [103]:
print(len(pb_achieved_feature_train_df_pandas))
print(len(pb_achieved_target_train_df_pandas))
print(len(pb_achieved_feature_test_df_pandas))
print(len(pb_achieved_target_test_df_pandas))

34098
34098
14614
14614


In [117]:
print(len(pb_achieved_feature_train_df_pandas.columns))
print(len(pb_achieved_target_train_df_pandas.columns))
print(len(pb_achieved_feature_test_df_pandas.columns))
print(len(pb_achieved_target_test_df_pandas.columns))

47
1
47
1
